# neuroNumber
### Version 0.1
The one hidden layer model for a machine learning

In [2]:
import numpy
from scipy.special import expit
import matplotlib.pyplot as plt

In [3]:
class Neural:
    
    def __init__(self, 
                 input_nodes, hidden_nodes, output_nodes, 
                 learn_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.lr = learn_rate
        
        self.wih = numpy.random.normal(
            0.0, 
            pow(self.hidden_nodes, -0.5),
            (self.hidden_nodes, self.input_nodes)
        )
        self.who = numpy.random.normal(
            0.0, 
            pow(self.output_nodes, -0.5),
            (self.output_nodes, self.hidden_nodes)
        )
        
        self.activation_fn = lambda x: expit(x)
    
    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_outputs, final_outputs = self.get_outputs(inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)),
            numpy.transpose(hidden_outputs)
            )
        self.wih += self.lr * numpy.dot(
            (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)),
            numpy.transpose(inputs)
            )

    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        _, final_outputs = self.get_outputs(inputs)
        
        return final_outputs
    
    
    def get_outputs(self, inputs):
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_fn(hidden_inputs)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_fn(final_inputs)
        
        return hidden_outputs, final_outputs

In [ ]:
# Training and test data get from here:
# https://pjreddie.com/projects/mnist-in-csv/
# and pun in workdir
# 
# ../
# neuro_number.ipynb
# mnist_test.csv
# mnist_train.csv

In [ ]:
training_file_name = "mnist_train.csv"
input_nodes = 784
hidden_nodes = 150
output_nodes = 10
learning_rate = 0.1
epochs = 5

with open(training_file_name, 'r') as training_data_file:
    training_data_list = training_data_file.readlines()
    

n = Neural(input_nodes, hidden_nodes, output_nodes, 
           learning_rate)

max_count = len(training_data_list)
n.data_length = max_count
n.training_file_name = training_file_name
n.epochs = epochs

print("=== TRAINING ===")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length}")
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes = {n.hidden_nodes}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")

for e in range(epochs):
    print(f"\n Epoch {e + 1} of {epochs}")
    count = 0
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        if count%5000 == 0:
            print(f"[{count} of {max_count}] Training ...")
        n.train(inputs, targets)
        count +=1
    
print("\n=== FINISH ===")


In [6]:
test_file_name = "mnist_test.csv"

with open(test_file_name, 'r') as testing_data_file:
    testing_data_list = testing_data_file.readlines()
    

max_count = len(testing_data_list)
correct = 0.50

count = 0
good = 0

print("=== TEST ===")
print(f"test file: {test_file_name}")
print(f"test data: {max_count} units")
print(f"correct value must be > {correct * 100}%")
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes = {n.hidden_nodes}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length}")
print(f"training epochs = {n.epochs}")
        
print()

for record in testing_data_list:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    if count%1000 == 0:
        print(f"[{count} of {max_count}] Testing ...")
    
    res = list(n.query(inputs))
    max_value = max(res)
    idx = res.index(max_value)
    number = int(all_values[0])
    if (float(max_value) > correct) and idx == number:
        good += 1
        
    count +=1

print("\n--- Report ---:")
print(f"Correct values is {good} from {max_count} ({good/max_count * 100:.3f}%)")
print("\n=== FINISH ===")

=== TEST ===
test file: mnist_test.csv
test data: 10000 units
correct value must be > 50.0%

--- Model ---
input nodes = 784
hidden nodes = 150
output nodes = 10
learning rate = 0.1
training file = mnist_train.csv
training data length = 60000
training epochs = 5

[0 of 10000] Testing ...
[1000 of 10000] Testing ...
[2000 of 10000] Testing ...
[3000 of 10000] Testing ...
[4000 of 10000] Testing ...
[5000 of 10000] Testing ...
[6000 of 10000] Testing ...
[7000 of 10000] Testing ...
[8000 of 10000] Testing ...
[9000 of 10000] Testing ...

--- Report ---:
Correct values is 9575 from 10000 (95.750%)

=== FINISH ===


### Efficiency ~95-96%